#Dataset preparation

Preparing the dataset for the purpose of the analyzing the players behavior and engagement with the game

In [1]:
import pandas as pd
import numpy as np

COD = pd.read_csv('/kaggle/input/games-1/cod.csv')
CS_GO = pd.read_csv('/kaggle/input/games-2/csgo_steam_reviews.csv')
Killzone_shadow = pd.read_csv('/kaggle/input/games-dataset/killzone_shadowfall_reviews.csv')
Killzone_shadow = Killzone_shadow.dropna()

In [12]:

COD.head(3)


,name,wins,kills,kdRatio,killstreak,level,losses,prestige,hits,timePlayed,headshots,averageTime,gamesPlayed,assists,misses,xp,scorePerMinute,shots,deaths
0,RggRt45#4697369,0,0,0.00000,0,1,0,0,0,0,0,0.0,0,0,0,0,0.0,0,0
1,JohniceRex#9176033,0,0,0.00000,0,1,0,110,0,7,0,7.0,0,0,0,700,0.0,0,16
2,bootybootykill#1892064,0,66,1.03125,0,9,0,110,0,32,16,32.0,0,1,0,48300,0.0,0,64


Now, applying RNN model for review ratings of FPS games

In [ ]:
pip install keras

In [9]:
pip install jaxlib==0.4.13

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install keras==2.13.1

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install jax==0.4.13

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install tensorflow==2.13.0

Note: you may need to restart the kernel to use updated packages.


# GRU model
Only for sentiment Analysis

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

# Load the player reviews data
reviews_df = pd.read_csv("/kaggle/input/games-2/csgo_steam_reviews.csv")  # Replace "player_reviews.csv" with your dataset file name

# Preprocess the data (assuming "review_text" column contains the review text and "rating" column contains the ratings)
reviews_df["review"] = reviews_df["review"].astype(str)  # Convert review_text to strings
reviews_text = reviews_df["review"].values
ratings = reviews_df["author_playtime_at_review"].values

# Convert ratings to integers
ratings = (ratings >= 0.5).astype(int)

# Tokenize and pad the review texts
max_sequence_length = 100  # Maximum sequence length for padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews_text)
sequences = tokenizer.texts_to_sequences(reviews_text)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, ratings, test_size=0.3, random_state=42, stratify=ratings)

# Define the GRU-based sentiment analysis model
embedding_dim = 100  # Dimensionality of word embeddings
hidden_size = 64  # Number of units in GRU layer
output_size = 1  # Binary classification (positive/negative sentiment)

model = Sequential()
model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_sequence_length))
model.add(GRU(hidden_size))
model.add(Dense(output_size, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the GRU model
batch_size = 64
num_epochs = 10

model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, validation_split=0.2, shuffle=True)

# Predict sentiment (positive/negative) on the test set
y_pred_prob = model.predict(X_test)
threshold = 0.5
y_pred = (y_pred_prob >= threshold).astype(int).flatten()

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

# Create a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)



Epoch 1/10
  2/137 [..............................] - ETA: 6s - loss: 0.6915 - accuracy: 0.5703

/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


137/137 [==============================] - 7s 53ms/step - loss: 0.0766 - accuracy: 0.9936 - val_loss: 6.6755e-05 - val_accuracy: 1.0000
Epoch 2/10
137/137 [==============================] - 7s 52ms/step - loss: 6.2197e-05 - accuracy: 1.0000 - val_loss: 5.7649e-05 - val_accuracy: 1.0000
Epoch 3/10
137/137 [==============================] - 8s 55ms/step - loss: 5.3168e-05 - accuracy: 1.0000 - val_loss: 4.8878e-05 - val_accuracy: 1.0000
Epoch 4/10
137/137 [==============================] - 7s 53ms/step - loss: 4.4869e-05 - accuracy: 1.0000 - val_loss: 4.1061e-05 - val_accuracy: 1.0000
Epoch 5/10
137/137 [==============================] - 7s 52ms/step - loss: 3.7547e-05 - accuracy: 1.0000 - val_loss: 3.4223e-05 - val_accuracy: 1.0000
Epoch 6/10
137/137 [==============================] - 7s 53ms/step - loss: 3.1280e-05 - accuracy: 1.0000 - val_loss: 2.8587e-05 - val_accuracy: 1.0000
Epoch 7/10
137/137 [==============================] - 7s 52ms/step - loss: 2.6349e-05 - accuracy: 1.0000 - va

# Playtime and compare reviews model
Now building the model to find the highest playingtime and then check what are the average playtime for negative and positive reviews

In [47]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [64]:
import pandas as pd
from prettytable import PrettyTable

# Load the player reviews data
reviews_df = pd.read_csv("/kaggle/input/games-2/csgo_steam_reviews.csv")  # Replace "player_reviews.csv" with your dataset file name

# Function to convert minutes to hours
def mins_to_hours(minutes):
    try:
        minutes = int(minutes)
        return f"{minutes // 60}h {minutes % 60}m"
    except ValueError:
        return "Invalid"

# Convert playing time from minutes to hours
reviews_df["author_playtime_at_review"] = reviews_df["author_playtime_at_review"].apply(mins_to_hours)

# Sort the DataFrame by "author_playtime_at_review" in descending order
highest_playing_time_players = reviews_df.sort_values(by="author_playtime_at_review", ascending=True)

# Select the top 10 players with the highest playing time
highest_playing_time_players = highest_playing_time_players.head(20)

# Display top highest playing time and reviews table with PrettyTable
table = PrettyTable()
table.field_names = ["Playing Time", "Review"]
for _, row in highest_playing_time_players.iterrows():
    table.add_row([row["author_playtime_at_review"], row["review"]])

print("Top 10 Players with Highest Playing Time:")
print(table)


Top 10 Players with Highest Playing Time:
+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Playing Time |                                                                                                                                               Review                                                                                                                                              |
+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    0h 10m    |               

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Advertising cost for 2024: 333.5
Predicted sales for 2024: 13667.846973365617
